In [1]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage
import plotly.express as px
import pandas as pd

# create clients
client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)


In [ ]:
#### GET react, angular, vue.js QUESTIONS ##############

sample_size = 100000

query = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE REGEXP_CONTAINS(tags, r"vue") OR REGEXP_CONTAINS(tags, r"react") OR REGEXP_CONTAINS(tags, r"angular")
     OR REGEXP_CONTAINS(tags, r"svelte")  OR REGEXP_CONTAINS(tags, r"ember.js")  OR REGEXP_CONTAINS(tags, r"jquery")
    ORDER BY r
    LIMIT ''' + str(sample_size)


df = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

df

In [ ]:
##### FRONTEND 1 CHANGE TAG, SAVE TO CSV ################

def setTag(x):
    if(x.find('react') != -1):
        return 'react'
    elif(x.find('angular') != -1):
        return 'angular'
    elif(x.find('vue.js') != -1):
        return 'vue.js'
    elif(x.find('ember.js') != -1):
        return 'ember.js'
    elif(x.find('svelte') != -1):
        return 'svelte'
    elif(x.find('jquery') != -1):
        return 'jquery'
    

toSaveDf = pd.DataFrame(df)
toSaveDf = toSaveDf.drop(columns=['r'])
toSaveDf.rename(columns={'tags':'framework'}, inplace=True)
toSaveDf['framework'] = toSaveDf['framework'].map(setTag)
toSaveDf.to_csv('js-framework-tags-dataframe.csv')

In [ ]:
## FRONTEND FRAMEWORK 2 ##
sample_size = 100000

query = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE ( REGEXP_CONTAINS(tags, r"html") 
        OR REGEXP_CONTAINS(tags, r"javascript")
        OR REGEXP_CONTAINS(tags, r"HTML")
        OR REGEXP_CONTAINS(tags, r"css")
        OR REGEXP_CONTAINS(tags, r"CSS"))
    AND NOT REGEXP_CONTAINS(tags, r"vue")
    AND NOT REGEXP_CONTAINS(tags, r"angular")
    AND NOT REGEXP_CONTAINS(tags, r"react")
    AND NOT REGEXP_CONTAINS(tags, r"express")
    AND NOT REGEXP_CONTAINS(tags, r"ember")
    AND NOT REGEXP_CONTAINS(tags, r"node")
    ORDER BY r
    LIMIT ''' + str(sample_size)

query_new = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE REGEXP_CONTAINS(tags, r"vue") OR REGEXP_CONTAINS(tags, r"react") OR REGEXP_CONTAINS(tags, r"angular")
    OR REGEXP_CONTAINS(tags, r"svelte")  OR REGEXP_CONTAINS(tags, r"ember")
    ORDER BY r
    LIMIT ''' + str(sample_size)


df_frontend2_old = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

df_frontend2_new = (
    client.query(query_new)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

print(df_frontend2_old)
print(df_frontend2_new)

In [ ]:
##### FRONTEND 2 CHANGE TAG, SAVE TO CSV ################

def setTag(x):
    if(
        x.find('react') != -1 or
        x.find('angular') != -1 or
        x.find('ember') != -1 or
        x.find('vue') != -1 or
        x.find('svelte') != -1
    ):
        return 'new'
    else:
        return 'old'
    
df_frontend2_new
df_frontend2_old
toSaveDf = df_frontend2_new.append(df_frontend2_old, ignore_index=True)
toSaveDf = toSaveDf.drop(columns=['r'])
toSaveDf.rename(columns={'tags':'technology'}, inplace=True)
toSaveDf['technology'] = toSaveDf['technology'].map(setTag)
toSaveDf.to_csv('old_vs_new_frontend.csv')


In [ ]:
toSaveDf

In [28]:
## BACKEND FRAMEWORK 1 ##
sample_size = 50000

query_back = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE (REGEXP_CONTAINS(tags, r"asp")
        OR REGEXP_CONTAINS(tags, r"php")
        OR REGEXP_CONTAINS(tags, r"ruby")
        OR REGEXP_CONTAINS(tags, r"servlet"))
    AND NOT REGEXP_CONTAINS(tags, r"django")
    AND NOT REGEXP_CONTAINS(tags, r"express")
    AND NOT REGEXP_CONTAINS(tags, r"spring")
    AND NOT REGEXP_CONTAINS(tags, r"node")
    AND NOT REGEXP_CONTAINS(tags, r"symfony")
    AND NOT REGEXP_CONTAINS(tags, r"ruby-on-rails")
    ORDER BY r
    LIMIT ''' + str(sample_size)

query_back_new = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE REGEXP_CONTAINS(tags, r"django") OR REGEXP_CONTAINS(tags, r"express") OR REGEXP_CONTAINS(tags, r"spring")
    OR REGEXP_CONTAINS(tags, r"node") OR REGEXP_CONTAINS(tags, r"symfony") OR REGEXP_CONTAINS(tags, r"ruby-on-rails")
    ORDER BY r
    LIMIT ''' + str(sample_size)


df_back_old = (
    client.query(query_back)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

df_back_new = (
    client.query(query_back_new)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

print(df_back_old)
print(df_back_new)

                  r                                        tags  \
0      2.481845e-07                                         php   
1      7.304773e-07                             css|asp.net-mvc   
2      1.202438e-06             javascript|php|jquery|html|ajax   
3      1.767459e-06           javascript|asp.net-mvc|dom-events   
4      2.097141e-06                                         php   
...             ...                                         ...   
49995  2.270987e-02                                   php|email   
49996  2.270997e-02              entity-framework|asp.net-mvc-4   
49997  2.271117e-02  php|header|opencart|schema.org|opencart2.x   
49998  2.271130e-02                              javascript|php   
49999  2.271159e-02                      php|variables|for-loop   

                         creation_date  
0     2020-04-25 03:19:40.707000+00:00  
1     2013-05-24 03:20:02.670000+00:00  
2     2015-06-10 08:42:06.827000+00:00  
3     2019-03-14 20:20:39.30000

In [31]:
##### BACKEND 2 CHANGE TAG, SAVE TO CSV ################

def setTag(x):
    if(
        x.find('django') != -1 or
        x.find('express') != -1 or
        x.find('spring') != -1 or
        x.find('node') != -1 or
        x.find('ruby-on-rails') != -1 or
        x.find('symfony') != -1
    ):
        return 'new'
    else:
        return 'old'
    
df_back_new
df_back_old
toSaveDf_back = df_back_new.append(df_back_old, ignore_index=True)
toSaveDf_back = toSaveDf_back.drop(columns=['r'])
toSaveDf_back.rename(columns={'tags':'technology'}, inplace=True)
toSaveDf_back['technology'] = toSaveDf_back['technology'].map(setTag)
#toSaveDf_back.to_csv('old_vs_new_backend.csv')

In [32]:
toSaveDf_back

,technology,creation_date
0,new,2012-07-18 18:30:00.077000+00:00
1,new,2016-10-20 16:09:58.737000+00:00
2,new,2013-07-24 03:48:41.130000+00:00
3,new,2012-01-18 23:29:54.517000+00:00
4,new,2015-07-31 21:24:51.373000+00:00
...,...,...
99995,old,2014-02-13 10:32:50.267000+00:00
99996,old,2013-02-12 10:55:45.663000+00:00
99997,old,2016-03-07 04:40:53.127000+00:00
99998,old,2017-12-02 15:24:09.777000+00:00


In [33]:

def setTagBack(x):
    return "back";
def setTagFront(x):
    return "front";


toSaveDf_back['technology'] = toSaveDf_back['technology'].map(setTagBack)


toSaveDf_front = pd.read_csv('js-framework-tags-dataframe.csv')
toSaveDf_front['framework'] = toSaveDf_front['framework'].map(setTagFront)

toSaveDf_back


,technology,creation_date
0,back,2012-07-18 18:30:00.077000+00:00
1,back,2016-10-20 16:09:58.737000+00:00
2,back,2013-07-24 03:48:41.130000+00:00
3,back,2012-01-18 23:29:54.517000+00:00
4,back,2015-07-31 21:24:51.373000+00:00
...,...,...
99995,back,2014-02-13 10:32:50.267000+00:00
99996,back,2013-02-12 10:55:45.663000+00:00
99997,back,2016-03-07 04:40:53.127000+00:00
99998,back,2017-12-02 15:24:09.777000+00:00


In [34]:
toSaveDf_front.rename(columns={'framework':'technology'}, inplace=True)

toSaveDf_front

,Unnamed: 0,technology,creation_date
0,0,front,2018-09-10 08:22:30.697000+00:00
1,1,front,2012-09-03 19:13:07.620000+00:00
2,2,front,2014-05-15 02:48:34.943000+00:00
3,3,front,2016-04-20 11:28:03.967000+00:00
4,4,front,2013-02-05 11:41:33.790000+00:00
...,...,...,...
99995,99995,front,2012-01-23 11:15:02.890000+00:00
99996,99996,front,2016-11-22 19:52:53.617000+00:00
99997,99997,front,2014-04-07 19:46:00.343000+00:00
99998,99998,front,2019-12-03 10:10:16.097000+00:00


In [35]:
toSaveDf_back

,technology,creation_date
0,back,2012-07-18 18:30:00.077000+00:00
1,back,2016-10-20 16:09:58.737000+00:00
2,back,2013-07-24 03:48:41.130000+00:00
3,back,2012-01-18 23:29:54.517000+00:00
4,back,2015-07-31 21:24:51.373000+00:00
...,...,...
99995,back,2014-02-13 10:32:50.267000+00:00
99996,back,2013-02-12 10:55:45.663000+00:00
99997,back,2016-03-07 04:40:53.127000+00:00
99998,back,2017-12-02 15:24:09.777000+00:00


In [36]:
toSaveDf_bvf = toSaveDf_front.append(toSaveDf_back, ignore_index=True)



toSaveDf_bvf.to_csv('back_vs_front.csv')

In [37]:
toSaveDf_bvf

,Unnamed: 0,technology,creation_date
0,0.0,front,2018-09-10 08:22:30.697000+00:00
1,1.0,front,2012-09-03 19:13:07.620000+00:00
2,2.0,front,2014-05-15 02:48:34.943000+00:00
3,3.0,front,2016-04-20 11:28:03.967000+00:00
4,4.0,front,2013-02-05 11:41:33.790000+00:00
...,...,...,...
199995,NaN,back,2014-02-13 10:32:50.267000+00:00
199996,NaN,back,2013-02-12 10:55:45.663000+00:00
199997,NaN,back,2016-03-07 04:40:53.127000+00:00
199998,NaN,back,2017-12-02 15:24:09.777000+00:00
